In [9]:
"""
file name:  main.ipynb
author:     Weili An
email:      an107@purdue.edu
version:    v1.0.0
desc:       main evalution file for Nvidia Sparsity training
"""
# Nvidia Automatic Sparsity 
# from apex.contrib.sparsity import ASP
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from tqdm import tqdm

In [2]:
"""
Dataset Loader
"""
# CIFAR10 dataset
# Preprocess ops from https://pytorch.org/hub/pytorch_vision_alexnet/
cifar10_preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
cifar10_trainset = torchvision.datasets.CIFAR10(root='./dataset', train=True,
                                        download=True, transform=cifar10_preprocess)
cifar10_trainloader = torch.utils.data.DataLoader(cifar10_trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

cifar10_testset = torchvision.datasets.CIFAR10(root='./dataset', train=False,
                                       download=True, transform=cifar10_preprocess)
cifar10_testloader = torch.utils.data.DataLoader(cifar10_testset, batch_size=4,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
"""
Model Loader
"""
# Load AlexNet
model_alexnet = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=False)
# model_alexnet.eval()

# Convert to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model_alexnet.to(device)

Using cache found in /home/an107/.cache/torch/hub/pytorch_vision_v0.6.0


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [8]:
# AlexNet Training setting
model = model_alexnet
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# AlexNet Training
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in tqdm(enumerate(cifar10_trainloader, 0)):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.623
[1,  4000] loss: 1.851
[1,  6000] loss: 1.627
[1,  8000] loss: 1.530
[1, 10000] loss: 1.429
[1, 12000] loss: 1.351
[2,  2000] loss: 1.225
[2,  4000] loss: 1.154
[2,  6000] loss: 1.120
[2,  8000] loss: 1.071
[2, 10000] loss: 1.032
[2, 12000] loss: 0.994
Finished Training


In [13]:
# AlexNet Eval
model = model_alexnet
correct = 0
total = 0
with torch.no_grad():
    for data in tqdm(cifar10_testloader):
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

100%|██████████| 2500/2500 [00:10<00:00, 233.82it/s]Accuracy of the network on the 10000 test images: 65 %



In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

cuda:0
